<a href="https://colab.research.google.com/github/smannan/LIFXBulbAnalysis/blob/main/SmartBulbDataExtractFromInflux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Query LIFX bulb data from an Influx DB client

1.   LIFX API spec: https://api.developer.lifx.com/docs/list-lights
2.   Visualize bulb brightness and power over time
3.   Will need Influx acces token, org, and bucket
4.   For info on Influx python client: https://docs.influxdata.com/influxdb/cloud/api-guide/client-libraries/python/

In [1]:
pip install influxdb-client

In [2]:
POWER_ENUM = { 'on': 1, 'off': 0 }

In [3]:
import pandas as pd

Create Influxe DB client

In [4]:
from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision

# You can generate a Token from the "Tokens Tab" in the UI
# https://us-central1-1.gcp.cloud2.influxdata.com/orgs/a33284f1a3112d0e/data-explorer
token = "BMNyseUoGWt6hU5tHxgYh6jyEkAYXAX514CIUtWhHmdu9nshYBAk75Oc1YBJOp84WyFL_4HIxdgkSkTpXvl_NA=="
org = "praveenkumar23.anguru@gmail.com"
bucket = "SmartBulbData"

client = InfluxDBClient(url="https://us-central1-1.gcp.cloud2.influxdata.com", token=token, verify_ssl=False)

In [5]:
### Print number of tables, records per table, and example record for each
### table returned by a query
def print_table_stats(tables):
  # one table per light bulb
  print ('Num tables {0}\n'.format(len(tables)))
  for table in tables:
    print ('Num records in table {0}'.format(len(table.records)))
    if len(table.records) > 0:
      record = table.records[0]
      print ('Sample record {0}, {1}\n'.format(record.get_field(), record.get_value()))

In [18]:
### Query a week's worth of data from an Influx db bucket for a specific field
### Get both brightness and kelvin fields
### Shift 7 hours in the future to compensate for utc default
"""
Each flux query returns a stream of tables meaning your query can return multiple tables.
Each table is created depending on the grouping. (|> group())
If you change the grouping at the end of your query you could merge these tables into 1.
https://stackoverflow.com/questions/67617181/why-is-this-influxdb-flux-query-returning-2-tables
Cannot group fields together if they're different data types?
"""
def query_field():
  query = f'from(bucket: \"{bucket}\") |> range(start: -55d) |> timeShift(duration: -7h) |> filter(fn:(r) => r._field =~ /brightness|kelvin/) |> group(columns: ["_field"])'
  print (query)
  tables = client.query_api().query(query, org=org)
  print_table_stats(tables)
  return tables

Get a week's worth of power data from the database

In [19]:
# table_id = 0 or 1 depending on which bulb
### Put the each record's value and time information into a pandas dataframe
def get_field_values_from_flux():
  tables = query_field()
  field_values = {}

  for table in tables:
    for record in table.records:
      value = record.get_value()
      field = record.values['_field']
      record_id = record.values['id']
      record_key = "{0}_{1}".format(record_id, record.get_time())

      row = {
          'timestamp': record.get_time(),
          'id': record_id
      }
      row[field] = value

      if record_key in field_values:
        field_values[record_key][field] = value
      else:
        field_values[record_key] = row

  dataframe = pd.DataFrame(list(field_values.values()))

  print ('Extracted {0} values'.format(len(dataframe)))
  print ('Earliest: {0}'.format(dataframe['timestamp'].min()))
  print ('Latest: {0}'.format(dataframe['timestamp'].max()))
  
  return dataframe

In [20]:
formatted_df = get_field_values_from_flux()

from(bucket: "SmartBulbData") |> range(start: -55d) |> timeShift(duration: -7h) |> filter(fn:(r) => r._field =~ /brightness|kelvin/) |> group(columns: ["_field"])


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Num tables 2

Num records in table 17789
Sample record brightness, 1.0

Num records in table 17789
Sample record kelvin, 2700

Extracted 17789 values
Earliest: 2021-08-26 13:46:01+00:00
Latest: 2021-10-19 18:45:01+00:00


In [21]:
formatted_df.head()

,timestamp,id,brightness,kelvin
0,2021-08-26 13:46:01+00:00,d073d55df12f,1.0000,2700
1,2021-08-26 13:47:00+00:00,d073d55df12f,1.0000,2700
2,2021-08-26 13:57:01+00:00,d073d55df12f,0.7021,2700
3,2021-08-26 13:58:01+00:00,d073d55df12f,0.7021,2700
4,2021-08-26 13:59:01+00:00,d073d55df12f,0.7021,2700


In [22]:
min_time = str(formatted_df['timestamp'].min()).replace(" ", "-")
max_time = str(formatted_df['timestamp'].max()).replace(" ", "-")
formatted_df.to_csv('smart_bulb_data_from_influx_{0}_{1}.csv'.format(min_time, max_time))

In [23]:
formatted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17789 entries, 0 to 17788
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype                    
---  ------      --------------  -----                    
 0   timestamp   17789 non-null  datetime64[ns, tzlocal()]
 1   id          17789 non-null  object                   
 2   brightness  17789 non-null  float64                  
 3   kelvin      17789 non-null  int64                    
dtypes: datetime64[ns, tzlocal()](1), float64(1), int64(1), object(1)
memory usage: 556.0+ KB


In [24]:
formatted_df['brightness'].describe()

count    17789.000000
mean         0.846536
std          0.223434
min          0.000000
25%          0.833700
50%          0.913400
75%          1.000000
max          1.000000
Name: brightness, dtype: float64

In [25]:
formatted_df['kelvin'].describe()

count    17789.000000
mean      3082.494800
std        431.584751
min          0.000000
25%       2700.000000
50%       2700.000000
75%       3500.000000
max       3500.000000
Name: kelvin, dtype: float64

In [26]:
min_time

'2021-08-26-13:46:01+00:00'

In [27]:
max_time

'2021-10-19-18:45:01+00:00'